# Verification of the diagnosis by the RIBEIRO PRE-TRAINED Model 
  
- On this notebook we wiil look further into the stats of the RIBEIRO NN and confirm the results annonced by his team.  
- We will work over a part of the giant dataset of more thant 2 millions 12-lead ECG.  
- For that we take 30000 ECG that will construct our validation dataset. It represents aproximately 1.3% of the whole training dataset
- We will create 7 datasets, one for each cardiac disease and the last one if the patient is healthy.  
- Also, in order to understand why the prediction is not correct, a {disease}_prob dataset will be created and some of the ECG that causes problem to the machine will be printed.


# I. Determine diseases datasets  
  
First we analyse the 1.5%_dataset to find what are the index of the abnormal-ECG, we categorise them into 7 lists.  
Those lists will be limited to 2 thousands index, it means that our stats won't be perfect but they will approximate the real ones.


  ### I.1. Open datas and take only 30000 samples


Taking the annotated ECG in order to compare the prediction to the annotation at the end.  
We want 1 or 2 thousands ECG of each diseases, then 30000 datas looks alright to do so.

In [4]:
import numpy as np
import h5py
from tqdm import tqdm

TEST_SHAPE = 30000 
PATH_TO_GIANT = '/media/cotxetjordi/TOSHIBA EXT/Data_2/'

with h5py.File(PATH_TO_GIANT + "preprocessed_traces15pc.hdf5", "r") as f:
    i = 0
    signals = []
    id_exams = []
    ids = list(f["id_exam"][:60000])  #Only 40000 raw data needed

    for index,data in tqdm(enumerate(f["signal"])):

        id = ids[index]

        if id < 2200000: # Only takes exams with annotations available in annotations.csv
            
            signals.append(data)
            id_exams.append(id)
            i+=1
            
        if i == TEST_SHAPE:
            break
            
        
signals = np.array(signals)
id_exam = np.array(id_exams)
print("La taille du dataset est de :", np.shape(signals))
print(len(id_exams))


40635it [00:07, 5340.05it/s]


La taille du dataset est de : (30000, 4096, 12)
30000


### I.2.a) Save the datas to use it later 


In [6]:
with h5py.File(PATH_TO_GIANT + 'datas_to_verif.hdf5', 'a') as new_file:
    id_exam_data = new_file.create_dataset(name = "id_exam",data = id_exams)
    signals_data = new_file.create_dataset(name = "signal",data= = signals)
    

### I.2.b) Load them

In [5]:
import h5py
PATH_TO_GIANT = 'F:/Data_2/'

with h5py.File(PATH_TO_GIANT + 'datas_to_verif.hdf5', 'r') as new_file:
    id_exams = list(new_file["id_exam"][()])
    X = new_file["signal"][()]



## I.3. Create 7 datasets, one for each disease
### 3.1. Find the annotation of each signal of our dataset and divide it into 7 datasets 



In [41]:

import time
import csv
PATH_TO = 'F:/Data_2/' #Path to your csv file with only the columns
counter = -1
start_time = time.time()
with open(PATH_TO + 'annotations_processed.csv','r', newline='') as csvfile:
        COUNTER = 0
        i = 1
        carac = {"np":[],"1dAVB":[],"RBBB":[],"LBBB":[],"SB":[],"AF":[],"ST":[]}
        f = csv.reader(csvfile)
        indexes = list(carac.keys())
        
        
        for line in f:
            if i != 1:
                id_ex = int(line[0])

                
            if id_ex in id_exams:

                if "1" in line and line.count("1") == 1 : #Just taking the "mono-disease" ECG to analyse
                    disease_ids = carac[indexes[line.index("1")]]

                    if len(disease_ids) < 2000:
                        disease_ids.append(id_exams.index(id_ex))

                        COUNTER +=1

          
               
                elif len(carac["np"]) < 2000:
                    carac["np"].append(id_exams.index(id_ex))
                    COUNTER +=1
       
            if i % 50000 == 0:
                    print(COUNTER)
                    print("--- %s seconds ---" % (time.time() - start_time))            
            if COUNTER == 12000:
                break
                
            i += 1

           
print("--- %s seconds ---" % (time.time() - start_time)) 


['267', '0', '0', '0', '0', '0', '1']
19970
267
{'np': [19970, 16052, 17408, 1836, 12362, 8286], '1dAVB': [], 'RBBB': [], 'LBBB': [], 'SB': [], 'AF': [], 'ST': [19970]}
['397', '0', '0', '0', '0', '1', '0']
28673
397
{'np': [19970, 16052, 17408, 1836, 12362, 8286, 2990, 24080], '1dAVB': [], 'RBBB': [], 'LBBB': [], 'SB': [], 'AF': [28673], 'ST': [19970]}
['408', '1', '0', '0', '0', '0', '0']
5774
408
{'np': [19970, 16052, 17408, 1836, 12362, 8286, 2990, 24080], '1dAVB': [5774], 'RBBB': [], 'LBBB': [], 'SB': [], 'AF': [28673], 'ST': [19970]}
['545', '1', '0', '0', '0', '0', '0']
10917
545
{'np': [19970, 16052, 17408, 1836, 12362, 8286, 2990, 24080, 9350, 23956, 28106], '1dAVB': [5774, 10917], 'RBBB': [], 'LBBB': [], 'SB': [], 'AF': [28673], 'ST': [19970]}
['756', '0', '0', '0', '0', '0', '1']
8943
756
{'np': [19970, 16052, 17408, 1836, 12362, 8286, 2990, 24080, 9350, 23956, 28106, 1849, 11953, 11400, 19010], '1dAVB': [5774, 10917], 'RBBB': [], 'LBBB': [], 'SB': [], 'AF': [28673], 'ST': [

KeyboardInterrupt: 

### 3.1.bis Create the preprocessed csv annotations


In [15]:
import csv
with open('F:/Data_2/annotations.csv','r', newline='') as csvfile:
    with open('F:/Data_2/annotations_processed.csv','w',newline='') as fichiercsv:
        writer=csv.writer(fichiercsv)
        i = 0
        read = csv.reader(csvfile)
        for line in read:


                i+=1
                writer.writerow([line[0]]+line[4:10]) # Only useful datas for testing


### I.3.a) Save the datas

In [33]:
import numpy as np

for index, (dis,ids) in enumerate(carac.items()) :

    np.save(dis,ids)



### I.3.b) Load the 7 datasets

In [34]:
import numpy as np
carac = {"np":[],"1dAVB":[],"RBBB":[],"LBBB":[],"SB":[],"AF":[],"ST":[]}
for index,(key,ids) in enumerate(carac.items()):
    
    carac[key] = np.load(key+ ".npy")

    print(key,"has", len(carac[key]),"samples")
    



np has 941 samples
1dAVB has 13 samples
RBBB has 37 samples
LBBB has 15 samples
SB has 13 samples
AF has 20 samples
ST has 18 samples


### I.4 Create an array composed by 7 arrays of disease-acquisition

In [35]:
X_test_all = np.array([[X[index] for index in ids] for ids in carac.values()]) #don't forget to convert each of the 7 lists when you will predict using the pre-trained model

    

## II. Prediction test
### II.1 Load and compile the model

In [2]:

# %% Import packages
import tensorflow.compat.v1 as tf  #Fait appel à la binliothèque Tensorflow Version 1.xx
tf.disable_v2_behavior()
import numpy as np
import warnings
import argparse

warnings.filterwarnings("ignore")
import keras
from keras.models import load_model
from keras.optimizers import Adam
import h5py

PATH_TO_MODEL  = 'C:/Users/valer/Desktop/Stage_Notebook/automatic-ecg-diagnosis-master/model/'



# Import model
model = keras.models.load_model(PATH_TO_MODEL+ 'model.hdf5',compile=False)

#model = load_model(PATH_TO_MODEL, compile=False)
model.compile(loss='binary_crossentropy', optimizer=Adam(),metrics = [tf.keras.metrics.BinaryAccuracy()])


print("Model saved")

Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Using TensorFlow backend.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model saved


### II.2. Predict

In [141]:
predictions = []
import math
from scipy.special import softmax

prediction = softmax(model.predict(np.array([X[12]]))*10*8)
print(id_exams[12])
u = model.predict(np.array([X[12]]))*10**8
sum = 0
for i in u :
    sum += np.exp(i)

sof = np.exp(u)
print(sof)
print(sum([i for i in sof]))

print(softmax(u))
"""for X in X_test_all:
    prediction = model.predict(np.array([X[0]]),verbose = 1)"""
print(prediction)
with h5py.File(PATH_TO_GIANT + 'datas_to_verif.hdf5', 'r') as new_file:
    for i in range(200):
        if not (y_final[new_file["id_exam"][i]] != np.array([0,0,0,0,0,0])).all():
            print(new_file["id_exam"][i])
            print(y_final[new_file["id_exam"][i]] )
            break


754189
[[2.47979460e+03 1.70598520e+00 1.06970521e+00 2.39693951e+00
  3.32541585e+00 1.02417362e+00]]


TypeError: 'numpy.ndarray' object is not callable

In [77]:
from tqdm import tqdm
results = []
Y_test_all = [[[0]*6 for i in range(len(carac["np"] ) ) ] , [[1,0,0,0,0,0]for i in range(len(carac["1dAVB"] ) ) ] , [[0,1,0,0,0,0]for i in range(len(carac["RBBB"] ) ) ] , [[0,0,1,0,0,0]for i in range(len(carac["LBBB"] ) ) ], [[0,0,0,1,0,0]for i in range(len(carac["SB"] ) ) ] , [[1,0,0,0,1,0]for i in range(len(carac["AF"] ) ) ] , [[1,0,0,0,0,1]for i in range(len(carac["ST"] ) ) ] ]
for ind,X in tqdm(enumerate(X_test_all)): 
    results.append(model.evaluate(np.array(X), np.array(Y_test_all[ind]), batch_size=128,verbose = 1))

0it [00:00, ?it/s]

941/941 [==============================] - 3s 3ms/step


1it [00:03,  3.34s/it]

37/37 [==============================] - 0s 3ms/step


3it [00:03,  1.06it/s]

13/13 [==============================] - 0s 5ms/step


5it [00:03,  2.03it/s]

18/18 [==============================] - 0s 4ms/step


7it [00:03,  1.82it/s]


In [3]:
y_score = model.predict(np.array(X_test_all[0]),verbose =1)

NameError: name 'X_test_all' is not defined

In [73]:
print(results)


[[0.03973692699902623, 0.41001826524734497], [1.870348334312439, 0.409746378660202], [2.414590835571289, 0.4033203125], [2.774968147277832, 0.40038779377937317], [2.66457462310791, 0.39788055419921875], [4.382474422454834, 0.39933207631111145], [5.364100933074951, 0.40066224336624146]]


In [81]:
import csv
import numpy as np
PATH_TO= "F:/Data_2/"

y_final = []

with open(PATH_TO + 'annotations_clean.csv','r', newline='') as csvfile:
    f = csv.reader(csvfile)
    o=0
    print("ok")
    for i in f:
        if o != 0:

            y_final.append([int(z) for z in i])
            
        o+=1
y_final = np.array(y_final)
print(y_final[15])
        

ok
[0 0 1 0 0 0]
